In [7]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


model = torch.load("model-final2", map_location=device)
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [1]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('DeepPavlov/rubert-base-cased-sentence', model_max_length=512)

In [2]:
import glob
import os
import numpy as np
import pickle
import re

labels = []
sentences = []
for file in glob.glob('./2021thesis/txt/*.txt', recursive=True):
    header =""
    print(file)
    with open(file, 'r') as f:
        count = 0
        sents = re.split(r'\.\s', f.read())
        for sentence in sents:
            if "Краткая информация" in sentence:
                header = 0
            elif "Диагностика" in sentence:
                header = 1
            elif "Лечение" in sentence:
                header = 2
            elif "Реабилитация" in sentence:
                header = 3
            elif "Профилактика" in sentence:
                header = 4
            if header != '':
                labels.append(header)
                sentences.append(sentence.replace('\n', ' '))

./2021thesis/txt/КР472.txt
./2021thesis/txt/КР8.txt
./2021thesis/txt/КР116.txt
./2021thesis/txt/КР77.txt
./2021thesis/txt/КР76.txt
./2021thesis/txt/КР62.txt
./2021thesis/txt/КР329.txt
./2021thesis/txt/КР467.txt
./2021thesis/txt/КР249.txt
./2021thesis/txt/КР507.txt
./2021thesis/txt/КР277.txt
./2021thesis/txt/КР288.txt
./2021thesis/txt/КР317.txt
./2021thesis/txt/КР129.txt
./2021thesis/txt/КР48.txt
./2021thesis/txt/КР75.txt
./2021thesis/txt/КР289.txt
./2021thesis/txt/КР504.txt
./2021thesis/txt/КР266.txt
./2021thesis/txt/КР460.txt
./2021thesis/txt/КР138.txt
./2021thesis/txt/КР71.txt
./2021thesis/txt/КР58.txt
./2021thesis/txt/КР70.txt
./2021thesis/txt/КР111.txt
./2021thesis/txt/КР139.txt
./2021thesis/txt/КР475.txt
./2021thesis/txt/КР515.txt
./2021thesis/txt/КР501.txt
./2021thesis/txt/КР529.txt
./2021thesis/txt/КР339.txt
./2021thesis/txt/КР66.txt
./2021thesis/txt/КР73.txt
./2021thesis/txt/КР67.txt
./2021thesis/txt/КР98.txt
./2021thesis/txt/КР476.txt
./2021thesis/txt/КР338.txt
./2021thesis/tx

In [3]:
tokens = [tokenizer([s], truncation=True, padding=True) for s in sentences]

In [8]:
import os
for i, value in enumerate(zip(labels, tokens)):
    label, token = value
    input_ids = torch.tensor(token.input_ids).to(device)
    attention_mask = torch.tensor(token.attention_mask).to(device)
    with open(str(label)+".enc", "a+") as f:
        for v in model.bert(input_ids, attention_mask=attention_mask, output_hidden_states = True).hidden_states[0][0][-1].detach().numpy():
            f.write(str(v)+" ")
        f.write("\n")
        f.write(sentences[i]+"\n")